<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/20202/aed_20202_112_bst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile bst.h
#pragma once

#include <functional>

using std::function;


template<typename T, typename R=T>
class Bst {
    struct Node {
        T       elem;
        Node*   left;
        Node*   right;

        Node(T elem, Node* left=nullptr, Node* right=nullptr)
            : elem(elem), left(left), right(right) {}
    };
    
    using fkey  = function<R(T)>;
    using fproc = function<void(T)>;
    using uint  = unsigned int;

    Node*   root;
    uint    len;
    fkey    key;
public:
    Bst(fkey key=[](T a) { return a; }): root(nullptr), len(0), key(key) {}
    ~Bst()                  { destroy(root); }

    void add(T elem)        { add(root, elem); }
    void inOrder(fproc f)   { inOrder(root, f); }

private:
    void destroy(Node* node) {
        if (node != nullptr) {
            destroy(node->left);
            destroy(node->right);
            delete node;
        }
    }
    void add(Node*& node, T elem) {
        if (node == nullptr) {
            node = new Node(elem);
            ++len;
        } else if (key(elem) < key(node->elem)) {
            add(node->left, elem);
        } else {
            add(node->right, elem);
        }
    }
    void inOrder(Node* node, fproc f) {
        if (node != nullptr) {
            inOrder(node->left, f);
            f(node->elem);
            inOrder(node->right, f);
        }
    }
};

Overwriting bst.h


In [14]:
%%writefile 1.cpp
#include <iostream>
#include <stdlib.h>
#include <time.h>
#include "bst.h"

using namespace std;

int main() {
    Bst<float>* bst1 = new Bst<float>();

    for (int i = 0; i < 20; ++i) {
        auto num = rand() % 100;
        cout << num << " ";
        bst1->add(num);
    }
    cout << endl;

    bst1->inOrder([](float x) { cout << x << " "; });
    cout << endl;

    delete bst1;

    return 0;
}

Overwriting 1.cpp


In [15]:
!g++ -std=c++17 -O2 -Wall 1.cpp && ./a.out

83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 
15 21 26 26 27 35 36 40 49 59 62 63 72 77 83 86 86 90 92 93 
